### Setup

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
import tldextract
from urllib.parse import urlparse, parse_qs
import re

### Dataset

In [2]:
df = pd.read_csv('../datasets/malicious_phish.csv')
df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [3]:
X = df.iloc[:, 0]
y = df.iloc[:, 1]

print(X.head())
print(y.head())
print(y.value_counts())

0                                     br-icloud.com.br
1                  mp3raid.com/music/krizz_kaliko.html
2                      bopsecrets.org/rexroth/cr/1.htm
3    http://www.garage-pirenne.be/index.php?option=...
4    http://adventure-nicaragua.net/index.php?optio...
Name: url, dtype: object
0      phishing
1        benign
2        benign
3    defacement
4    defacement
Name: type, dtype: object
type
benign        428103
defacement     96457
phishing       94111
malware        32520
Name: count, dtype: int64


In [4]:
def classification_type(type):
    '''
    Convert classification type into values:
        1) Benign = 0
        2) Defacement = 1
        3) Phishing = 2
        4) Malware = 3    
    '''
    if type == "benign":
        return 0
    elif type == "defacement":
        return 1
    elif type == "phishing":
        return 2
    elif type == "malware":
        return 3
    else:
        print(f"Unable to find proper type: {type}")

y = y.apply(classification_type)
y.value_counts()

type
0    428103
1     96457
2     94111
3     32520
Name: count, dtype: int64

### Feature extraction

Extract any features not reliant on a lookup

#### URL

In [5]:
# Length
def get_url_length(url):
    return len(url)

# Number of ; _ ? = &
def get_url_punct_count(url):
    return len(re.findall(r'[;_\?=&]', url))

# Digit to Letter ratio
def get_dlr(url):
    num_digits = len(re.findall(r'\d', url))
    num_letters = len(re.findall(r'[a-zA-Z]', url))
    if num_letters:
        return num_digits / num_letters
    else:
        return 0

def get_url_feat(X_):
    return pd.concat([
        X_.apply(get_url_length),       # Length
        X_.apply(get_url_punct_count),  # Number of ; _ ? = &
        X_.apply(get_dlr)],             # Digit to Letter ratio
    axis=1)

# URL_feat = get_url_feat(X)
# URL_feat.head()

#### Primary domain

In [6]:
def get_primary_domain(url):
    extracted = tldextract.extract(url)
    return extracted.domain + '.' + extracted.suffix

# Presence of IP
def get_PD_has_IP(url):
    return int(bool(re.match(r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}', url)))

# Length
def get_PD_len(url):
    return len(url)

# Number of digits
def get_PD_digits(url):
    return len(re.findall(r'\d', url))

# Number of non-alphanumeric characters
def get_PD_num_non_alphanumeric(url):
    return len(re.findall(r'[^a-zA-Z0-9]', url))

# Number of hyphens
def get_PD_num_hyphens(url):
    return len(re.findall(r'-', url))

# Number of @s
def get_PD_num_at(url):
    return len(re.findall(r'@', url))

def get_PD_feat(X_):
    PD = X_.apply(lambda x: get_primary_domain(x))
    return pd.concat([
        PD.apply(get_PD_has_IP),               # Presence of IP
        PD.apply(get_PD_len),                  # Length
        PD.apply(get_PD_digits),               # Number of digits
        PD.apply(get_PD_num_non_alphanumeric), # Number of non-alphanumeric characters
        PD.apply(get_PD_num_hyphens),          # Number of hyphens
        PD.apply(get_PD_num_at)],              # Number of @s
        axis=1)

# PD_feat = get_PD_feat(X)
# PD_feat.head()

#### Subdomain

In [7]:
# Number of dots
def get_SD_num_dots(url):
    return url.count('.')

# Number of subdomains
def get_num_SD(url):
    return sum(1 for subdomain in url.split('.') if subdomain)

def get_SD_feat(X_):
    SD = X_.apply(lambda x: tldextract.extract(x).subdomain)
    return pd.concat([
        SD.apply(get_SD_num_dots),  # Number of dots
        SD.apply(get_num_SD)],      # Number of subdomains
        axis=1)

# SD_feat = get_SD_feat(X)
# SD_feat.head()

#### Path

In [8]:
# ensure proper parsing of paths
def parse_path(url):
    try:
        if url.startswith('https://') or url.startswith('http://'):
            return urlparse(url)
        else:
            return urlparse('https://' + url)
    except:
        return urlparse(url)

# Number of //
def get_path_num_dbl_fwdslash(url):
    return len(re.findall(r'//', url))

# Number of subdirectories
def get_path_num_subdirs(url):
    return url.count('/')

# Presence of %20
def get_path_has_percent20(url):
    return int('%20' in url)

# Presence of uppercase directories
def get_path_has_uppercase_dirs(url):
    return int(any(any(c.isupper() for c in dir_name) for dir_name in url.split('/')))

# Presence of single character directories
def get_path_has_char_dirs(url):
    return int(any((len(dir_name) == 1) for dir_name in url.split('/')))

# Number of special characters
def get_path_num_special_chars(url):
    return sum(len(re.findall(r'[^A-Za-z0-9]', dir_name)) for dir_name in url.split('/'))

# Number of 0s
def get_path_num_zeroes(url):
    return url.count('0')

# Ratio of uppercase to lowercase characters
def get_path_upper_to_lower_ratio(url):
    upper_count = sum(1 for c in url if c.isupper())
    lower_count = sum(1 for c in url if c.islower())
    if lower_count:
        return upper_count / lower_count
    else:
        return 0

def get_path_feat(X_):
    path_ = X_.apply(lambda x: parse_path(x).path)
    return pd.concat([
        path_.apply(get_path_num_dbl_fwdslash),         # Number of //
        path_.apply(get_path_num_subdirs),              # Number of subdirectories
        path_.apply(get_path_has_percent20),            # Presence of %20
        path_.apply(get_path_has_uppercase_dirs),       # Presence of uppercase directories
        path_.apply(get_path_has_char_dirs),            # Presence of single character directories
        path_.apply(get_path_num_special_chars),        # Number of special characters
        path_.apply(get_path_num_zeroes),               # Number of 0s
        path_.apply(get_path_upper_to_lower_ratio)],    # Ratio of uppercase to lowercase characters
        axis=1)

# path_feat = get_path_feat(X)
# path_feat.head()

#### Parameters

In [9]:
# Length
def get_param_length(url):
    return sum(len(value[0]) for value in parse_qs(urlparse(url).query).values())

def get_param_feat(X_):
    return pd.concat([
        X_.apply(get_param_length)],    # Length
        axis=1)

# param_feat = get_param_feat(X)
# param_feat.head()

#### Query

In [10]:
# Number of queries
def get_num_queries(url):
    return len(parse_qs(urlparse(url).query))

def get_query_feat(X_):
    return pd.concat([
        X_.apply(get_num_queries)
    ])

# query_feat = get_query_feat(X)
# query_feat.head()

In [11]:
def get_lexical_feature_set(X_):
    return pd.concat([
        get_url_feat(X_),
        get_PD_feat(X_),
        get_SD_feat(X_),
        get_path_feat(X_),
        get_param_feat(X_),
        get_query_feat(X_)],
        axis=1)

feature_set = get_lexical_feature_set(X)
feature_set.head()

,url,url,url,url,url,url,url,url,url,url,...,url,url,url,url,url,url,url,url,url,url
0,16,0,0.000000,0,16,0,3,1,0,0,...,0,0,0,0,0,0,0,0.0,0,0
1,35,1,0.034483,0,11,1,1,0,0,0,...,0,2,0,0,0,2,0,0.0,0,0
2,31,0,0.040000,0,14,0,1,0,0,0,...,0,3,0,0,0,1,0,0.0,0,0
3,88,10,0.111111,0,17,0,2,1,0,0,...,0,1,0,0,0,1,0,0.0,22,4
4,235,7,0.110553,0,23,0,2,1,0,0,...,0,1,0,0,0,1,0,0.0,175,3


### Train Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(feature_set, y, test_size=0.2, random_state=69)

In [13]:
print(y_train.value_counts())
print(y_test.value_counts())

type
0    342451
1     77182
2     75393
3     25926
Name: count, dtype: int64
type
0    85652
1    19275
2    18718
3     6594
Name: count, dtype: int64


### Model

In [14]:
model = RandomForestClassifier(n_estimators=50, max_depth=20, min_samples_split=2, min_samples_leaf=2)

In [15]:
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=50)

### Evaluation

In [16]:
def calc_FNR_accuracy(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    for label_class in range(4):
        FN = sum(conf_matrix[label_class][i] for i in range(len(conf_matrix)) if i != label_class)  
        
        TP = conf_matrix[label_class][label_class]  
        
        TN = np.sum(np.delete(np.delete(conf_matrix, label_class, axis=0), label_class, axis=1))
        
        accuracy = (TP + TN) / np.sum(conf_matrix)
        print("Accuracy for class", label_class, ":", accuracy)

        FNR = FN / (FN + TP) if (FN + TP) > 0 else -1
        print("FNR for class", label_class, ":", FNR)

#### Malicious URLS (multi-class)
1) Benign       = 0
2) Defacement   = 1
3) Phishing     = 2
4) Malware      = 3 

In [17]:
y_pred = model.predict(X_test)

In [18]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_test, y_pred)

In [19]:
print(f'OVERALL: Accuracy: {accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, f1 Score: {f1:.8f}')

OVERALL: Accuracy: 0.89893964, Precision: 0.89774210, Recall: 0.89893964, f1 Score: 0.89605304


In [20]:
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')

Class 0:	Test Precision: 0.90312575,	Test Recall: 0.96544155,	Test f1: 0.93324455
Class 1:	Test Precision: 0.90275271,	Test Recall: 0.83029831,	Test f1: 0.86501095
Class 2:	Test Precision: 0.83384636,	Test Recall: 0.68020088,	Test f1: 0.74922765
Class 3:	Test Precision: 0.99454225,	Test Recall: 0.85668790,	Test f1: 0.92048232


In [21]:
calc_FNR_accuracy(y_test, y_pred)

Accuracy for class 0 : 0.9091669929898111
FNR for class 0 : 0.03455844580395087
Accuracy for class 1 : 0.9616474327966278
FNR for class 1 : 0.16970168612191958
Accuracy for class 2 : 0.9345587727178495
FNR for class 2 : 0.3197991238380169
Accuracy for class 3 : 0.9925060849668687
FNR for class 3 : 0.14331210191082802


#### External Datasets

In [27]:
# benign
df_benign = pd.read_csv('../datasets/benign_urls.csv').iloc[:, 0]
df_benign.head()

0       https://www.google.com
1      https://www.youtube.com
2     https://www.facebook.com
3        https://www.baidu.com
4    https://www.wikipedia.org
Name: url, dtype: object

In [28]:
y_pred_benign = model.predict(get_lexical_feature_set(df_benign))
y_test_benign = np.full((df_benign.shape[0],), 0)

In [29]:
benign_accuracy = accuracy_score(y_test_benign, y_pred_benign)
benign_precision = precision_score(y_test_benign, y_pred_benign, average='weighted')
benign_recall = recall_score(y_test_benign, y_pred_benign, average='weighted')
benign_f1 = f1_score(y_test_benign, y_pred_benign, average='weighted')

c:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
print(f'OVERALL: Accuracy: {benign_accuracy:.8f}, Precision: {benign_precision:.8f}, Recall: {benign_recall:.8f}, f1 Score: {benign_f1:.8f}')

OVERALL: Accuracy: 0.75573700, Precision: 1.00000000, Recall: 0.75573700, f1 Score: 0.86087723


In [31]:
calc_FNR_accuracy(y_test_benign, y_pred_benign)

Accuracy for class 0 : 0.7557370031642457
FNR for class 0 : 0.24426299683575425
Accuracy for class 1 : 0.874529267827083
FNR for class 1 : -1
Accuracy for class 2 : 0.8815490342398001
FNR for class 2 : -1
Accuracy for class 3 : 0.9996587010973628
FNR for class 3 : -1


___

In [32]:
# phishing
df_phish = pd.read_csv('../datasets/phishtank_phish_urls.csv').iloc[:, 0]
df_phish.head()

0                       https://servicecu.mobi/Service
1          https://sp581716.sitebeat.crazydomains.com/
2          https://sp774647.sitebeat.crazydomains.com/
3          https://sp779562.sitebeat.crazydomains.com/
4    https://manaliindiancuisine.es//gmx/rechnung/d...
Name: url, dtype: object

In [33]:
y_pred_phish = model.predict(get_lexical_feature_set(df_phish))
y_test_phish = np.full((df_phish.shape[0],), 0)

In [34]:
phish_accuracy = accuracy_score(y_test_phish, y_pred_phish)
phish_precision = precision_score(y_test_phish, y_pred_phish, average='weighted')
phish_recall = recall_score(y_test_phish, y_pred_phish, average='weighted')
phish_f1 = f1_score(y_test_phish, y_pred_phish, average='weighted')

c:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
print(f'OVERALL: Accuracy: {phish_accuracy:.8f}, Precision: {phish_precision:.8f}, Recall: {phish_recall:.8f}, f1 Score: {phish_f1:.8f}')

OVERALL: Accuracy: 0.74563949, Precision: 1.00000000, Recall: 0.74563949, f1 Score: 0.85428806


In [36]:
calc_FNR_accuracy(y_test_phish, y_pred_phish)

Accuracy for class 0 : 0.7456394858473061
FNR for class 0 : 0.2543605141526939
Accuracy for class 1 : 0.9774467797982525
FNR for class 1 : -1
Accuracy for class 2 : 0.7695084511318782
FNR for class 2 : -1
Accuracy for class 3 : 0.9986842549171755
FNR for class 3 : -1
